In [7]:
import pybotics as py
import numpy as np
import scipy.optimize
import scipy.stats

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

# Init Robot

In [9]:
# load UR10 model for testing
model = np.array([
    0, 0, 0, 118,
    1.5707963267949, 0, 3.14159265358979, 0,
    0, 612.7, 0, 0,
    0, 571.6, 0, 163.9,
    - 1.5707963267949, 0, 0, 115.7,
    1.5707963267949, 0, 3.14159265358979, 92.2
]).reshape((-1, 4))
robot = py.Robot(model)

print('Robot Model:\n{}'.format(robot.robot_model))

Robot Model:
[[   0.       0.       0.     118.   ]
 [   1.571    0.       3.142    0.   ]
 [   0.     612.7      0.       0.   ]
 [   0.     571.6      0.     163.9  ]
 [  -1.571    0.       0.     115.7  ]
 [   1.571    0.       3.142   92.2  ]]


# Init Tool

In [10]:
nominal_tool_xyz = [100, 100, 50]
real_tool_xyz = [99.123, 100.987, 51.192]

# "Real" Measures

In [11]:
# simulate real measures (this would actually be externally measured data)
robot.set_tool_xyz(real_tool_xyz)

joints = []
torques = []
positions = []
for i in range(100):
    robot.random_joints()
    joints.append(robot.joint_angles)
    torques.append([0] * robot.num_dof())
    positions.append(robot.fk()[:-1, -1])


# Calibrate Tool

## Compute Initial Errors

In [12]:
# use nominal tool as seed for calibration
robot.set_tool_xyz(nominal_tool_xyz)

initial_errors = py.calibration.compute_absolute_errors(robot=robot,
                                                        joints=joints,
                                                        torques=torques,
                                                        positions=positions,
                                                        reference_frame=robot.world_frame
                                                        )
print('Initial tool:\n{}'.format(robot.tool.tcp))
print('Initial errors:\n{}'.format(scipy.stats.describe(initial_errors)))

Initial tool:
[[   1.    0.    0.  100.]
 [   0.    1.    0.  100.]
 [   0.    0.    1.   50.]
 [   0.    0.    0.    1.]]
Initial errors:
DescribeResult(nobs=100, minmax=(1.7795090709430685, 1.7795090709432604), mean=1.7795090709431611, variance=9.2519788284247579e-28, skewness=-0.03484375749420982, kurtosis=1.2081119707559465)


## Calibrate Using Least Squares

In [13]:
# set calibration mask
tool_mask = [True] * 6
tool_mask[3] = False
tool_mask[4] = False
tool_mask[5] = False

optimization_mask = robot.generate_optimization_mask(tool_mask=tool_mask)
optimization_vector = robot.generate_optimization_vector(optimization_mask)

# calibrate
optimize_result = scipy.optimize.leastsq(func=py.calibration.calibration_fitness_func,
                                         x0=optimization_vector,
                                         args=(
                                             optimization_mask,
                                             robot,
                                             joints,
                                             torques,
                                             positions,
                                             robot.world_frame,
                                             'abs',
                                             'list'
                                         ))

# process results
calibrated_tool = optimize_result[0]
print('Calibrated tool:\n{}'.format(calibrated_tool))

Calibrated tool:
[  99.123  100.988   51.193]
